# Bees vs Wasps notebook

#### Imports

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# additional classic imports
from pathlib import Path
import pandas as pd
import numpy as np
import random
import os
import gc
import cv2


from tensorflow.keras.layers import *
from keras.optimizers import Adam
from tqdm import tqdm

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras
##Selle järgi võiks teha https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
##https://www.kaggle.com/koshirosato/bee-or-wasp-base-line-using-resnet50/notebook

Using TensorFlow backend.


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU') 
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
# pip3 install --upgrade tensorflow-gpu

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17582731878813570253
]


#### Run parameters

In [4]:
bs = 64 # Batch size
resize_size = 96 # for training, resize all the images to a square of this size
training_subsample = 0.1 # for development, use a small fraction of the entire dataset rater than full dataset
ROOT = './data'
IMG_SIZE = 256
EPOCHS = 50

In [5]:
bees_vs_wasps_dataset_path=Path(ROOT) # this is relative to the "example_notebook" folder. Modify this to reflect your setup
df_labels = pd.read_csv(bees_vs_wasps_dataset_path/'labels.csv')
df_labels=df_labels.set_index('id')
# perform dataset subsampling
df_labels = df_labels.sample(frac=training_subsample, axis=0)

In [6]:
for idx in tqdm(df_labels.index):    
    df_labels.loc[idx,'path']=df_labels.loc[idx,'path'].replace('\\', '/') 
    
df_labels.head()

100%|████████████████████████████████████████████████████████████████████████████| 1142/1142 [00:00<00:00, 2819.77it/s]


,path,is_bee,is_wasp,is_otherinsect,is_other,photo_quality,is_validation,is_final_validation,label
id,,,,,,,,,
9129,other_insect/3421542693_9dd1487959_m.jpg,0,0,1,0,1,0,1,insect
2798,bee2/NP7466-94r.jpg,1,0,0,0,0,0,0,bee
223,bee1/14715854799_ca7323ebcd_n.jpg,1,0,0,0,1,0,1,bee
2105,bee1/7581111696_086e92baca_w.jpg,1,0,0,0,1,0,0,bee
5951,wasp2/F00455.jpg,0,1,0,0,0,0,0,wasp


## Andmestikude loomine




In [7]:
#Andmete kättesaamine algse csv järgi
train_df = df_labels.query('is_validation == 0 & is_final_validation == 0').reset_index(drop=True)
val_df = df_labels.query('is_validation == 1').reset_index(drop=True)
test_df = df_labels.query('is_final_validation == 1').reset_index(drop=True)

In [8]:
#Meetod admestikude loomiseks pildi failidega
def create_datasets(df, img_size):
    imgs = []
    for path in tqdm(df['path']):
        img = cv2.imread(ROOT+"/"+path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_size,img_size))
        imgs.append(img)

    imgs = np.array(imgs, dtype='float32')
    imgs = imgs / 255.0
    df = pd.get_dummies(df['label'])
    return imgs, df


train_imgs, train_df = create_datasets(train_df, IMG_SIZE)
val_imgs, val_df = create_datasets(val_df, IMG_SIZE)
test_imgs, test_df = create_datasets(test_df, IMG_SIZE)

100%|███████████████████████████████████████████████████████████████████████████████| 167/167 [00:01<00:00, 127.50it/s]


In [9]:
#Närvivõrk
def make_model(img_size, n):
    inp = Input(shape=(img_size,img_size,n))
    resnet = tf.keras.applications.ResNet50(input_shape=(img_size,img_size,n), 
                                            weights='imagenet', 
                                            include_top=False)
    x = resnet(inp)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(4, activation='softmax')(x)
    model = tf.keras.Model(inputs=inp, outputs=x) 
    opt = tf.keras.optimizers.SGD(momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

    model = Model(inputs=x, outputs=p)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    model.summary()
    return model

model = make_model(IMG_SIZE, 3)

In [10]:
print(train_imgs.shape)
print(train_df.shape)

(808, 256, 256, 3)
(808, 4)


In [ ]:

history = model.fit(train_imgs, 
                    train_df, 
                    batch_size=bs, 
                    epochs=EPOCHS,
                    validation_data=(val_imgs, val_df)
                   )

Epoch 1/50
 1/13 [=>............................] - ETA: 11:09 - loss: 1.7776 - accuracy: 0.2188

In [ ]:
train_df